Data Generation

In [1]:
import numpy as np
from geomdl import BSpline
from geomdl import utilities
from geomdl import operations
from geomdl import exchange
from geomdl import shapes
from geomdl import construct
from geomdl.visualization import VisPlotly as vis
from geomdl import CPGen
from matplotlib import cm
import random
import time

In [2]:
#Helper Functions

#from https://stackoverflow.com/questions/8487893/generate-all-the-points-on-the-circumference-of-a-circle
def PointsInCircum(r,z,n=15):
    return [[np.cos(2*np.pi/n*x)*r,np.sin(2*np.pi/n*x)*r, float(z)] for x in range(0,n + 1)]

Generate Geometry

In [3]:
numLevels = 5
numPtsOnLevel = 6
stepBetweenLevels = 3
radius = 5
offset = 1

In [37]:
#Initialize vertices of bottle

mouthVerticesInner = [PointsInCircum(2 ,0,numPtsOnLevel)]
topVerticesInner = [PointsInCircum(radius,(numLevels + 1) * stepBetweenLevels,numPtsOnLevel)]

variableVerticesInner = []

#this is what needs to be changed
offsetList = []

if len(offsetList) != numLevels * numPtsOnLevel:
    print("Incorrect number of points! Will crash!")
    L = []
    L[0]

indexOffset = 0
for level in range(0 + 1, numLevels + 1):
    nextVertices = PointsInCircum(radius + offset,level * stepBetweenLevels,numPtsOnLevel)
    for i in range(len(nextVertices) - 1):
        factor = offsetList[indexOffset]
        nextVertices[i][0] *= factor
        nextVertices[i][1] *= factor
        if (i == 0): nextVertices[-1] = nextVertices[0]
        indexOffset += 1
    variableVerticesInner += [nextVertices]

allVerticesInner = mouthVerticesInner + variableVerticesInner + topVerticesInner

print("Successfully generated points and a unique tensor!")


Incorrect number of points! Will have unexpected results!


IndexError: list index out of range

Preview Geometry

In [6]:
# to test the perodic curve

curve = BSpline.Curve()
curve.degree= 3
curve.ctrlpts = allVerticesInner[0]
curve.knotvector = utilities.generate_knot_vector(curve.degree, curve.ctrlpts_size)

# tangent_vector = utilities.vector_generate(curve.ctrlpts[0], curve.ctrlpts[-1])  # Calculate the vector between start and end points
# curve.ctrlpts[1] = tuple(utilities.vector_add(curve.ctrlpts[0], tangent_vector))  # Set the modified control point 1
# curve.ctrlpts[-2] = tuple(utilities.vector_add(curve.ctrlpts[-1], tangent_vector))  # Set the modified control point 2

curve.vis = vis.VisCurve2D(ctrlpts=True, legend=False)
print("Generating visualization, please wait...")
curve.render(colormap=cm.terrain)

Generating visualization, please wait...


In [35]:
surf = BSpline.Surface()
surf.degree_u = 3
surf.degree_v = 3
surf.ctrlpts2d = allVerticesInner
surf.knotvector_u = utilities.generate_knot_vector(surf.degree_u, surf.ctrlpts_size_u)
surf.knotvector_v = utilities.generate_knot_vector(surf.degree_v, surf.ctrlpts_size_v)

surf.vis = vis.VisSurface(ctrlpts=True, legend=False)
print("Generating visualization, please wait...")
surf.render(colormap=cm.terrain)

Generating visualization, please wait...


Save Geometry & Tensor

In [10]:
mesh_text = ""
firstTriangle = True
for triangle in surf.faces:
    if not firstTriangle: mesh_text += "|"
    else: firstTriangle = False
    firstVertex = True
    for vertex in triangle:
        if not firstVertex: mesh_text += ","
        else: firstVertex = False
        mesh_text += str(vertex[0])
        mesh_text += ","
        mesh_text += str(vertex[1])
        mesh_text += ","
        mesh_text += str(vertex[2])

tensor_text = ""
firstFactor = False
for factor in offsetList:
    if not firstFactor: tensor_text += ","
    else: firstFactor = False
    tensor_text += str(factor)

seconds = time.time()

mesh_file = open(".\\mesh_data\\mesh_%s.text" % seconds, "w")
mesh_file.write(mesh_text)
mesh_file.close()


tensor_file = open(".\\tensor_data\\tensor_%s.text" % seconds, "w")
tensor_file.write(tensor_text)
tensor_file.close()